In [1]:
import pylab as pl
import numpy as np

import matplotlib as mpl
mpl.rcParams['font.sans-serif'] = ['SimHei']

### 拟合与优化-optimize
#### 非线性方程组求解

In [2]:
from scipy import optimize

def f(x):
    x0, x1, x2 = x.tolist()
    return [
        5*x1+3,
        4*x0*x0 - 2*np.sin(x1*x2),
        x1*x2 - 1.5]

result = optimize.fsolve(f, [1,1,1])
#Find the roots of a function.
result, f(result)

(array([-0.70622057, -0.6       , -2.5       ]),
 [0.0, -9.126033262418787e-14, 5.329070518200751e-15])

In [ ]:
def j(x): 
    x0, x1, x2 = x.tolist()
    return [[0, 5, 0],
            [8 * x0, -2 * x2 * np.cos(x1 * x2), -2 * x1 * np.cos(x1 * x2)],
            [0, x2, x1]]
result = optimize.fsolve(f, [1, 1, 1], fprime=j)

#### 最小二乘拟合
$r = arg\min(sum(func(y)^2, axis=0)$

In [4]:
X = np.array([ 8.19,  2.72,  6.39,  8.71,  4.7 ,  2.66,  3.78])
Y = np.array([ 7.01,  2.78,  6.47,  6.71,  4.1 ,  4.23,  4.05])

def residuals(p, y, x):
    "计算以p为参数的直线和原始数据之间的误差"
    k, b = p
    return Y - (k*X + b)

# leastsq使得residuals()的输出数组的平方和最小，参数的初始值为[1,0]
r = optimize.leastsq(residuals, [1, 0], args=(Y, X))
k, b = r[0]
print ("k =",k, "b =",b)

k = 0.6134953491930442 b = 1.794092543259387


#### 计算函数局域最小值

In [ ]:
def target_function(x, y):
    return (1 - x)**2 + 100 * (y - x**2)**2


class TargetFunction(object):
    def __init__(self):
        self.f_points = []
        self.fprime_points = []
        self.fhess_points = []

    def f(self, p):
        x, y = p.tolist()
        z = target_function(x, y)
        self.f_points.append((x, y))
        return z

    def fprime(self, p):
        x, y = p.tolist()
        self.fprime_points.append((x, y))
        dx = -2 + 2 * x - 400 * x * (y - x**2)
        dy = 200 * y - 200 * x**2
        return np.array([dx, dy])

    def fhess(self, p):
        x, y = p.tolist()
        self.fhess_points.append((x, y))
        return np.array([[2 * (600 * x**2 - 200 * y + 1), -400 * x],
                         [-400 * x, 200]])


def fmin_demo(method):
    target = TargetFunction()
    init_point = (-1, -1)
    res = optimize.minimize(
        target.f,
        init_point,
        method=method,
        jac=target.fprime,
        hess=target.fhess)
    return res, [
        np.array(points) for points in (target.f_points, target.fprime_points,
                                        target.fhess_points)
    ]

In [6]:
# 计算全域最小值
def func(x, p):
    A, k, theta = p
    return A*np.sin(2*np.pi*k*x+theta)

def func_error(p, y, x):
    return np.sum((y - func(x, p))**2)   

x = np.linspace(0, 2*np.pi, 100)
A, k, theta = 10, 0.34, np.pi/6 
y0 = func(x, [A, k, theta])
np.random.seed(0)
y1 = y0 + 2 * np.random.randn(len(x))

result = optimize.basinhopping(func_error, (1, 1, 1),
                      niter = 10,
                      minimizer_kwargs={"method":"L-BFGS-B",
                                        "args":(y1, x)})
print (result.x)

[10.25218676 -0.34239909  2.63341581]


### 线性代数-linalg

#### 解线性方程组

In [7]:
from scipy import linalg

m, n = 500, 50
A = np.random.rand(m, m)
B = np.random.rand(m, n)
X1 = linalg.solve(A, B)
X2 = np.dot(linalg.inv(A), B)
print (np.allclose(X1, X2))

True


In [8]:
luf = linalg.lu_factor(A)
X3 = linalg.lu_solve(luf, B)
np.allclose(X1, X3)

True

In [9]:
# 特征值和特征向量
A = np.array([[1, -0.3], [-0.1, 0.9]])
evalues, evectors = linalg.eig(A)

### 统计-stats

In [10]:
from scipy import stats
X = stats.norm(loc=1.0, scale=2.0)
X.stats()
x = X.rvs(size=10000) # 对随机变量取10000个值
stats.norm.fit(x) # 得到随机序列期望值和标准差
x = stats.gamma.rvs(2, scale=2, size=4)  
stats.gamma.pdf(x, 2, scale=2) #Probability density function at x.

array([0.10459777, 0.18372474, 0.18161843, 0.11908394])

#### 离散概率分布

In [11]:
x = range(1, 7)    
p = (0.4, 0.2, 0.1, 0.1, 0.1, 0.1)

dice = stats.rv_discrete(values=(x, p))
dice.rvs(size=20)

array([5, 1, 1, 1, 2, 5, 4, 5, 1, 2, 3, 3, 5, 1, 1, 2, 1, 1, 2, 2])

In [13]:
stats.binom.pmf(range(6), 5, 1/6.0)#Probability mass function at k 

array([4.01877572e-01, 4.01877572e-01, 1.60751029e-01, 3.21502058e-02,
       3.21502058e-03, 1.28600823e-04])